# 闭包

In [1]:
time = 0

def insert_time(min):
    time = time + min
    return  time

print(insert_time(2))
print(insert_time(10))

UnboundLocalError: local variable 'time' referenced before assignment

在 Python 中，如果一个函数使用和全局变量相同的名字且改变该变量的值，那么该变量就会变成局部变量，那么就会造成在函数中我们没有进行定义就引用了，所以会报错误。如果确实要引用全局变量，并在函数中对它进行修改，可以使用 `global` 关键字。

In [2]:
time = 0

def insert_time(min):
    global  time
    time = time + min
    return  time

print(insert_time(2))
print(insert_time(10))

2
12


全局变量降低了函数或模块之间的通用性，不同的函数或模块都要依赖于全局变量。这时候我们使用闭包来解决。

In [4]:
time = 0

def study_time(time):
    def insert_time(min):
        nonlocal  time
        time = time + min
        return time

    return insert_time # 函数也是对象，可以直接返回函数


f = study_time(time)
print(f(2))
print(time)
print(f(10))
print(time)

2
0
12
0


这里最直接的表现就是全局变量 time 至此至终都没有修改过，这里用了 `nonlocal` 关键字，表示在函数或其他作用域中使用外层(非全局)变量。

这种内部函数的局部作用域中可以访问外部函数局部作用域中变量的行为，我们称为闭包。更加直接的表达方式是，当某个函数被当成对象返回时，夹带了外部变量，就形成了一个闭包。

闭包避免了使用全局变量，此外，闭包允许将函数与其所操作的某些数据（环境）关连起来，而且使用闭包可以使代码变得更加的优雅。

有没有什么办法来验证一下这个函数就是闭包呢？有的，所有函数都有一个 `__closure__` 属性，如果函数是闭包的话，那么它返回的是一个由 cell 组成的元组对象。cell 对象的 cell_contents 属性就是存储在闭包中的变量。

In [5]:
time = 0

def study_time(time):
    def insert_time(min):
        nonlocal  time
        time = time + min
        return time

    return insert_time

f = study_time(time)
print(f.__closure__)
print(f(2))
print(time)
print(f.__closure__[0].cell_contents)
print(f(10))
print(time)
print(f.__closure__[0].cell_contents)

(<cell at 0x7fb4a3644790: int object at 0x10e62b8a0>,)
2
0
2
12
0
12


从结果可见，传进来的值一直存储在闭包的 cell_contents 中，这也是闭包的最大特点，可以将父函数的变量与其内部定义的函数绑定。就算生成闭包的父函数已经释放了，闭包仍然存在。

闭包的过程其实好比类（父函数）生成实例（闭包），不同的是父函数只在调用时执行，执行完毕后其环境就会释放，而类则在文件执行时创建，一般程序执行完毕后作用域才释放，因此对一些需要重用的功能且不足以定义为类的行为，使用闭包会比使用类占用更少的资源，且更轻巧灵活。

# 装饰器

In [2]:
import time

def punch():
    print(time.strftime('%Y-%m-%d', time.localtime(time.time())))
    print('昵称：两点水  部门：事业部 上班打卡成功')

punch()

2022-08-17
昵称：两点水  部门：事业部 上班打卡成功


这样改变了函数的功能结构，可能会造成代码重复的问题。可以使用**函数式编程**来修改这部分的代码。Python 函数有两个特点，函数也是一个对象，而且函数里可以嵌套函数。

In [8]:
import time

def punch():
    print('昵称：两点水  部门：事业部 上班打卡成功')

def add_time(func):
    print(time.strftime('%Y-%m-%d', time.localtime(time.time())))
    func()

add_time(punch)

2022-03-08
昵称：两点水  部门：事业部 上班打卡成功


使用函数编程很方便，但是每次调用的时候，都不得不把原来的函数作为参数传递进去，还能不能有更好的实现方式呢？有的，装饰器。

In [9]:
import time

def decorator(func):
    def punch():
        print(time.strftime('%Y-%m-%d', time.localtime(time.time())))
        func()

    return punch

def punch():
    print('昵称：两点水  部门：事业部 上班打卡成功')

f = decorator(punch)
f()

2022-03-08
昵称：两点水  部门：事业部 上班打卡成功


装饰器函数一般做这三件事：

1.接收一个函数作为参数

2.嵌套一个包装函数，包装函数会接收原函数的相同参数，并执行原函数，且还会执行附加功能

3.返回嵌套函数

Python 装饰器的核心可以说就是它的语法糖。那么怎么使用语法糖呢？根据上面的写法写完装饰器函数后，直接在原来的函数上加 @ 和装饰器的函数名。

In [10]:
import time

def decorator(func):
    def punch():
        print(time.strftime('%Y-%m-%d', time.localtime(time.time())))
        func()

    return punch

@decorator
def punch():
    print('昵称：两点水  部门：事业部 上班打卡成功')

punch()

2022-03-08
昵称：两点水  部门：事业部 上班打卡成功


In [11]:
import time

def decorator(func):
    def punch(*args, **kwargs):
        print(time.strftime('%Y-%m-%d', time.localtime(time.time())))
        func(*args, **kwargs)

    return punch

@decorator
def punch(name, department):
    print('昵称：{0}  部门：{1} 上班打卡成功'.format(name, department))

@decorator
def print_args(reason, **kwargs):
    print(reason)
    print(kwargs)


punch('两点水', '事业部')
print_args('两点水', sex='男', age=99)

2022-03-08
昵称：两点水  部门：事业部 上班打卡成功
2022-03-08
两点水
{'sex': '男', 'age': 99}
